<a href="https://colab.research.google.com/github/rakshit9/Generative-AI/blob/main/langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install "langchain>=0.3.0" "langgraph>=0.2.0" "langchain-openai>=0.2.0" "python-dotenv>=1.0.1"

In [ ]:
import os, getpass
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
print("API key set ✅")

In [ ]:
from typing import TypedDict, List
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import StateGraph, END


In [ ]:
# 1) Define the state carried through the graph
class GraphState(TypedDict, total=False):
    messages: List           # chat history (HumanMessage/AIMessage objects)
    thought: str             # model's reasoning text
    answer: str              # final concise answer

In [ ]:
# 2) Define your model (use a fast, cheap one)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)


In [ ]:
# 3) Define nodes (each node = one step)
def think_node(state: GraphState) -> GraphState:
    """The agent thinks about the question."""
    # get the latest user message
    user_msg = next((m for m in reversed(state.get("messages", [])) if isinstance(m, HumanMessage)), None)
    question = user_msg.content if user_msg else "No question"
    response = llm.invoke([
        SystemMessage(content="You are a helpful AI that explains reasoning clearly but concisely."),
        HumanMessage(content=f"Think step by step about this question, in 3–5 short bullets:\n\n{question}")
    ])
    return {
        "thought": response.content,
        "messages": [AIMessage(content=f"[Thought]\n{response.content}")]
    }


In [55]:
def answer_node(state: GraphState) -> GraphState:
    """The agent gives a clear final answer based on the thought."""
    thought = state.get("thought", "")
    response = llm.invoke([
        SystemMessage(content="You are a friendly tutor. Be direct and concise."),
        HumanMessage(content=f"Based on this reasoning:\n{thought}\n\nNow give a clear final answer in 3–5 sentences.")
    ])
    return {
        "answer": response.content,
        "messages": [AIMessage(content=f"[Answer]\n{response.content}")]
    }

In [ ]:
think_node({"messages": [HumanMessage(content="What is the capital of France?")]})

In [ ]:
answer_node({'thought': '- The question asks for the capital city of a specific country, France.\n- The capital city is typically the political and administrative center of a country.\n- France is a well-known country in Europe.\n- The capital of France is Paris.\n- Paris is recognized for its cultural, historical, and economic significance.',
 'messages': [AIMessage(content='[Thought]\n- The question asks for the capital city of a specific country, France.\n- The capital city is typically the political and administrative center of a country.\n- France is a well-known country in Europe.\n- The capital of France is Paris.\n- Paris is recognized for its cultural, historical, and economic significance.', additional_kwargs={}, response_metadata={})]})

In [ ]:

# 4) Build the graph (nodes + edges)
graph = StateGraph(GraphState)
graph.add_node("think", think_node)
graph.add_node("answer", answer_node)

In [ ]:

graph.set_entry_point("think")      # START here
graph.add_edge("think", "answer")   # then go to answer
graph.add_edge("answer", END)       # then stop

app = graph.compile()

In [ ]:
from langchain_core.messages import HumanMessage
def run_agent(question: str):
    initial_state: GraphState = {"messages": [HumanMessage(content=question)]}
    result = app.invoke(initial_state)
    print("🤔 Thought:\n", result.get("thought", "(no thought)"), "\n")
    print("💬 Final Answer:\n", result.get("answer", "(no answer)"))
    return result


print("Use run_agent('your question') to try it!")

In [ ]:
_ = run_agent("Why do we see lightning before we hear thunder?")